# **TRAINING THE MODEL - LBP FACIAL EMOTION**

#### IMPORTING LIBRARIES

In [1]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import layers, models
from tqdm import tqdm

#### LOADING 

In [2]:
# Load the Haar cascade file for face detection
face_cascade = cv2.CascadeClassifier(r'D:\MTECH\PROJECT\DM\SECRET\CASCADE\haarcascade_frontalface_default.xml')


# Path to the dataset directory
dataset_dir = r'C:\Users\rahul\Downloads\class-20230604T115655Z-001\class'

LIST EMOTIONS

In [3]:
# List of emotions
emotions = ['angerH', 'angerL', 'happyH', 'happyL', 'sadH', 'sadL', 'surprise']


# Number of orthogonal vectors to keep
num_vectors = 40

# List to store images and labels
images = []
labels = []



#### ITERATE OVER IMAGES

In [4]:
# Iterate over the images in the dataset
for emotion in emotions:
    emotion_dir = os.path.join(dataset_dir, emotion)
    image_files = os.listdir(emotion_dir)

    for image_file in tqdm(image_files, desc=f'Processing {emotion}'):
        image_path = os.path.join(emotion_dir, image_file)

        # Load the input image
        image = cv2.imread(image_path)

        # Convert the image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale image
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.5, minNeighbors=5, minSize=(30, 30))

        # Iterate over the detected faces
        for (x, y, w, h) in faces:
            # Extract the face region
            face = gray_image[y:y + h, x:x + w]

            # Resize the face image to 32x32 pixels
            resized_face = cv2.resize(face, (32, 32), interpolation=cv2.INTER_LINEAR)

            # Normalize the pixel values between 0 and 1
            normalized_face = resized_face / 255.0

            # Calculate LBP (Local Binary Patterns) values
            lbp = local_binary_pattern(resized_face, 8, 1, method='uniform')

            # Resize the LBP image to match the size of the normalized face image
            resized_lbp = cv2.resize(lbp, (32, 32), interpolation=cv2.INTER_LINEAR)

            # Flatten the normalized face and resized LBP images
            flattened_face = normalized_face.flatten()
            flattened_lbp = resized_lbp.flatten()

            # Concatenate the flattened face and LBP as the combined feature
            combined_feature = np.concatenate([flattened_face, flattened_lbp])

            # Check dimensions
            if combined_feature.shape[0] != 32 * 32 * 2:
                print(f"Combined feature size mismatch for {image_file}: {combined_feature.shape[0]}")

            # Append the combined feature and label to the images and labels lists
            images.append(combined_feature)
            labels.append(emotion)


Processing surprise: 100%|██████████| 885/885 [00:01<00:00, 858.12it/s]


Convert images and labels to NumPy arrays

In [5]:

# Convert images and labels to NumPy arrays
images = np.array(images)
labels = np.array(labels)


Encode emotion labels to integers

In [6]:

# Encode emotion labels to integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

#### Split the dataset into training and testing sets

In [7]:

# Split the dataset into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    images, encoded_labels, test_size=0.2, random_state=42
)


#### CNN

Reshape the input images for CNN

In [8]:
# Reshape the input images for CNN
train_images = train_images.reshape(-1, 32, 32, 2)
test_images = test_images.reshape(-1, 32, 32, 2)

Define the CNN model architecture

In [9]:

# Define the CNN model architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 2)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(emotions), activation='softmax'))


#### Compile the model

In [10]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Train the model

In [11]:
# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
120/120 [==============================] - 4s 23ms/step - loss: 1.9063 - accuracy: 0.2218
Epoch 2/10
120/120 [==============================] - 3s 24ms/step - loss: 1.6916 - accuracy: 0.3570
Epoch 3/10
120/120 [==============================] - 3s 28ms/step - loss: 1.2225 - accuracy: 0.5663
Epoch 4/10
120/120 [==============================] - 3s 25ms/step - loss: 0.7419 - accuracy: 0.7641
Epoch 5/10
120/120 [==============================] - 3s 25ms/step - loss: 0.3482 - accuracy: 0.9066
Epoch 6/10
120/120 [==============================] - 3s 24ms/step - loss: 0.1447 - accuracy: 0.9669
Epoch 7/10
120/120 [==============================] - 3s 24ms/step - loss: 0.0475 - accuracy: 0.9953
Epoch 8/10
120/120 [==============================] - 3s 22ms/step - loss: 0.0142 - accuracy: 0.9997
Epoch 9/10
120/120 [==============================] - 3s 22ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 10/10
120/120 [==============================] - 3s 24ms/step - loss: 0.0036 - accura

#### ACCURACY

Evaluate the model on the testing set

In [12]:
# Evaluate the model on the testing set
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)

30/30 [==============================] - 0s 6ms/step - loss: 0.1434 - accuracy: 0.9771
Test Loss: 0.1433616280555725


test_accuracy

In [13]:
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.9770594239234924


### CLASSIFICATION REPORT

Predict emotions for the testing set

In [14]:
# Predict emotions for the testing set
predicted_labels = model.predict(test_images)
predicted_labels = np.argmax(predicted_labels, axis=1)


30/30 [==============================] - 0s 7ms/step


Convert the predicted labels back to original emotion labels

In [15]:
# Convert the predicted labels back to original emotion labels
predicted_emotions = label_encoder.inverse_transform(predicted_labels)

Generate classification report

In [16]:
# Generate classification report
unique_labels = label_encoder.inverse_transform(np.unique(test_labels))
report = classification_report(test_labels, predicted_labels, target_names=unique_labels)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

      angerH       0.99      0.97      0.98       244
      angerL       0.95      0.98      0.97       115
      happyH       0.98      0.98      0.98       170
      happyL       0.98      1.00      0.99       117
        sadH       0.98      0.97      0.97       115
        sadL       0.96      1.00      0.98       107
    surprise       0.98      0.96      0.97        91

    accuracy                           0.98       959
   macro avg       0.97      0.98      0.98       959
weighted avg       0.98      0.98      0.98       959



#### **SAVE THE MODEL**

In [17]:
# Save the model
model.save('BEST_MODEL.h5')